# Solving the self-consistency equations

This notebook describes how the analaytical expressions for the self-consistency equations and stability condition are numerically solved.

In [42]:
import numpy as np
import pandas as pd
import os
import sys
from copy import deepcopy

os.chdir('C:/Users/jamil/Documents/PhD/GitHub projects/Ecological-Dynamics-and-Community-Selection/Ecological-Dynamics/Consumer-Resource Models/alternative_growth_consumption_coupling/cavity_solutions_vs_simulations')

sys.path.insert(0, 'C:/Users/jamil/Documents/PhD/Github Projects/Ecological-Dynamics-and-Community-Selection/Ecological-Dynamics/Consumer-Resource Models/cavity_method_functions')
import self_consistency_equation_functions as sce

sys.path.insert(0, "C:/Users/jamil/Documents/PhD/GitHub projects/Ecological-Dynamics-and-Community-Selection/" + \
                    "Ecological-Dynamics/Consumer-Resource Models/alternative_growth_consumption_coupling")
from simulation_functions import generate_simulation_df, le_pivot_r, pickle_dump

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


All solvers and self-consistency equations are included in the *self_consistency_equation_functions* module.

## Solving the self-consistency equations for a set of model parameters

To solve the self-consistency equations, we use a global solver that is not sensitive to initial conditions.
We supply the statistical properties of model parameters as fixed values, and ask the solver to solve for the self-consistency equations.

In [43]:
# parameter set
parameters = dict(M = 150, gamma = 1,
                  mu_c = 145, sigma_c = 1.6,
                  mu_y = 1, sigma_y = 0.130639,
                  mu_b = 1, sigma_b = 0,
                  mu_d = 1, sigma_d = 0)

# quantities being solved for - the self consistency equations
solved_quantities = ['phi_N', 'N_mean', 'q_N', 'v_N',
                     'phi_R', 'R_mean', 'q_R', 'chi_R']

# bounds on the values of the self-consistency equations
bounds = ([1e-10, 1e-10, 1e-10, -1e15, 1e-10, 1e-10,  1e-10, 1e-10],
          [1, 1e15, 1e15, 1e-10, 1, 1e15, 1e15, 1e15])

# initial guesses for the self consistency equations (not that important as we are calling a global solver)
initial_values = np.array([0.5, 0.01, 0.01, -0.1, 0.7, 0.001, 0.01, 0.05])

# Solve the self consistency equations
# unfortunately, the solver takes a very long time in jupyter notebook 
sol = sce.solve_self_consistency_equations(model = 'self-limiting, yc c', # determines what analytically-dervied self consistency equations should be used
                                           parameters = parameters,
                                           solved_quantities = solved_quantities,
                                           bounds = bounds,
                                           x_init = initial_values,
                                           solver_name = 'basin-hopping', # global solving routine
                                           solver_kwargs = {'xtol' : 1e-13, 'ftol' : 1e-13}, # arguments for the local solver (non-linear least squares)
                                           other_kwargs = {'niter' : 300}) # arguments for the global solver (basinhopping)


100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


*sol* is a dataframe containing the parameters, the solved self-consistency equations + the final value of the loss function, and instability equations from eq. (62) and (63) in the SI.

In [14]:
print(sol)

     M  gamma  mu_c  sigma_c  mu_y   sigma_y  mu_b  sigma_b  mu_d  sigma_d  \
0  150      1   145      1.6     1  0.130639     1        0     1        0   

   ...       q_N       v_N     phi_R    R_mean      q_R     chi_R       loss  \
0  ...  0.000226 -0.439658  0.639686  0.006829  0.00011  0.300955 -54.722754   

         dNde       dRde  ms_loss  
0 -192.269516 -36.712112 -27.0927  

[1 rows x 22 columns]


For more details on the *solve_self_consistency_equations* functions, please see:

In [36]:
print(sce.solve_self_consistency_equations.__doc__)


    
    Calls solve_sces for solving the system of self-consistency equations 
    (phi_N, N_mean, q_N, v_N, phi_R, R_mean, q_R, and chi_R) and the solve for
    the stability quations (dNde and dRde)

    Parameters
    ----------
    model : str
        Name of the system of self-consistency equations you want to solve, aka
        the model you're solving.
        The options are 'self-limiting, rho' (Blumenthal et al., 2024),
        'self-limiting, yc c' (our model), 'self-limiting, g cg' (consumption
         is coupled to growth), 'externally supplied' (chemostat-style system
         with non-reciprocol growth and consumption rates)
    parameters : list of dicts or dict
        Values that are not being solved for. Usually the statistical properties 
        of the distributions of model parameters (e.g. mean consumption rate)
    solved_quantities : list of str
        The names of the quantities being solved for. Usually the self-consistency
        equations.
    bounds :

## Solving for the stability boundary

To identify the stability boundary while varying a specific parameter, we employed only the local solver for computational efficiency.
We fix all statistical properties of the model parameters except the one being varied, and supplied them to the solver. 
The solver is instructed to solve both the self-consistency equations and the critical value of the varying parameter at which the stability threshold is satisfied ($\rho_{c, g} = S^*/M^*$).
To get the solver to satisfy the stability condition, we set its argument *include_multistability* condition to *True* to include the stability criterion in the loss function.
ince the local solver is sensitive to initial conditions, the initial values we supply it are the solution from our parameter set that yielded the highest instability measures (*dNde* and *dRde*).

Note that the code below is just an example for how the solve the stability boundary.
Do not expect it to solve for the boundary well, as the actual protocol we used required a lot of refining (of the initial values being supplied).

In [34]:
parameters_boundary = deepcopy(parameters)
parameters_boundary.pop("mu_c")

solved_quantities_boundary = solved_quantities + ["mu_c"]

bounds_boundary = ([1e-10, 1e-10, 1e-10, -1e15, 1e-10, 1e-10,  1e-10, 1e-10, 0.85 * parameters["mu_c"]],
                  [1, 1e15, 1e15, 1e-10, 1, 1e15, 1e15, 1e15, 1.15 * parameters["mu_c"]])

initial_values_boundary = np.append(sol.loc[:, solved_quantities].to_numpy(), parameters["mu_c"])

sol_stability = sce.solve_self_consistency_equations(model = 'self-limiting, yc c',
                                                     parameters = parameters_boundary,
                                                     solved_quantities = solved_quantities_boundary,
                                                     bounds = bounds_boundary,
                                                     x_init = initial_values_boundary,
                                                     solver_name = 'least-squares', # local solving routine
                                                     solver_kwargs = {'xtol' : 1e-13, 'ftol' : 1e-13},
                                                     include_multistability = True) # include stability condition in the loss function


100%|██████████| 1/1 [06:36<00:00, 396.26s/it]


In [35]:
print(sol_stability)

     M  gamma  sigma_c  mu_y   sigma_y  mu_b  sigma_b  mu_d  sigma_d  \
0  150      1      1.6     1  0.130639     1        0     1        0   

      phi_N  ...      v_N     phi_R   R_mean       q_R     chi_R        mu_c  \
0  0.007779  ... -0.35009  0.016373  0.00403  0.003548  0.012103  157.558154   

        loss       dNde      dRde    ms_loss  
0 -47.335327  191.05247  1.063717 -27.092673  

[1 rows x 22 columns]
